In [3]:
%run Imports.ipynb

In [4]:
conf = pyspark.SparkConf()
spark = SparkSession.builder.appName("Rank").getOrCreate()

In [5]:
df1=spark.read.parquet("/Users/yashasvi/Documents/CleanDuneData/*")

In [6]:
df1.count()

3898970

In [7]:
df1.createOrReplaceTempView("Load")

In [8]:
df_rule_1 = spark.sql("SELECT buyer, AVG(usd_amount) AS buying_power, COUNT(buyer) AS txn_buy FROM Load GROUP BY buyer")

In [9]:
df_rule_1.show()

+--------------------+------------------+-------+
|               buyer|      buying_power|txn_buy|
+--------------------+------------------+-------+
|\x4623b7a9eb459fd...|       2970.340307|     10|
|\xa3d773e4f9080ae...|466.26832299999995|      5|
|\x3c5b3a8287f759e...| 2.246813566666667|      3|
|\xb738397f0a03aa5...|         58.711685|      2|
|\x1386d7a006bd547...| 762.9974736110116|     31|
|\xc38d773623dbb7b...|1020.3994119999999|     12|
|\x2e43850af79290f...|         5.4629215|      4|
|\x4299b157abde3ef...|        153.630215|      2|
|\xfc2dd34349f9abd...|          674.5711|      3|
|\x08dd1e75fe3a46c...|     2222.25308322|     10|
|\x0cb540e8a237e57...|1207.6800986666665|      3|
|\x20f6db36863d7a5...| 560.1388333333333|      3|
|\x1bd93fb42ea6a03...| 678.9134799999999|      2|
|\x6b937c85f26d6b0...| 514.5364943333333|      6|
|\xa800f34505e8b34...|       6379.152486|      4|
|\x8ba659f0be00b59...|284.57286472403683|      6|
|\x09328f42f0bc356...|          39.09095|      2|


# holding query

In [13]:
df_holding = spark.sql("WITH buyer_table AS (SELECT buyer, nft_token_id, nft_contract_address, original_amount, usd_amount, block_time FROM load WHERE original_amount > 0.0 AND trade_type = 'Single Item Trade'), seller_table AS (SELECT seller, nft_token_id, nft_contract_address, original_amount, usd_amount, block_time FROM load WHERE original_amount > 0.0 AND trade_type = 'Single Item Trade') SELECT buyer_table.buyer as person, buyer_table.nft_token_id, buyer_table.original_amount as buy_price, seller_table.original_amount as sell_price, seller_table.original_amount - buyer_table.original_amount as eth_profit, seller_table.usd_amount - buyer_table.usd_amount as usd_profit, CASE WHEN (seller_table.block_time > buyer_table.block_time) THEN (seller_table.block_time - buyer_table.block_time) ELSE (buyer_table.block_time - seller_table.block_time) END AS time_difference, CASE WHEN (seller_table.block_time > buyer_table.block_time) THEN EXTRACT(days from (seller_table.block_time - buyer_table.block_time)) ELSE EXTRACT(days from (buyer_table.block_time - seller_table.block_time)) END AS interval_days,CASE WHEN (seller_table.block_time > buyer_table.block_time) THEN EXTRACT(hours from (seller_table.block_time - buyer_table.block_time)) ELSE EXTRACT(hours from (buyer_table.block_time - seller_table.block_time)) END AS interval_hours, buyer_table.block_time as buy_time, seller_table.block_time as sell_time FROM buyer_table INNER JOIN seller_table on buyer_table.buyer = seller_table.seller AND buyer_table.nft_token_id = seller_table.nft_token_id AND buyer_table.nft_contract_address = seller_table.nft_contract_address")

# rest of information query

In [14]:
df_all = spark.sql("SELECT * FROM ( SELECT n1.buyer, n1.original_currency, n1.nft_token_id, n1.nft_contract_address, n1.original_amount as Bought, n2.original_amount as Sold, n1.block_time as Bought_Timestamp, n2.block_time as Sold_Timestamp,n1.usd_amount as Bought_amount, n2.usd_amount as Sold_amount, (n2.usd_amount - n1.usd_amount) as net, n1.platform as n1platform, n2.platform as n2platform FROM Load n1, Load n2 where n1.buyer = n2.seller AND  n2.block_time > n1.block_time AND  n1.nft_token_id is not null AND n1.nft_token_id = n2.nft_token_id AND n1.nft_contract_address = n2.nft_contract_address) holdings")

In [15]:
df_all.columns

['buyer',
 'original_currency',
 'nft_token_id',
 'nft_contract_address',
 'Bought',
 'Sold',
 'Bought_Timestamp',
 'Sold_Timestamp',
 'Bought_amount',
 'Sold_amount',
 'net',
 'n1platform',
 'n2platform']

In [16]:
df_all.schema

StructType(List(StructField(buyer,StringType,true),StructField(original_currency,StringType,true),StructField(nft_token_id,StringType,true),StructField(nft_contract_address,StringType,true),StructField(Bought,DoubleType,true),StructField(Sold,DoubleType,true),StructField(Bought_Timestamp,TimestampType,true),StructField(Sold_Timestamp,TimestampType,true),StructField(Bought_amount,DoubleType,true),StructField(Sold_amount,DoubleType,true),StructField(net,DoubleType,true),StructField(n1platform,StringType,true),StructField(n2platform,StringType,true)))

In [19]:
type(df_all)

pyspark.sql.dataframe.DataFrame

In [20]:
df_all.select('buyer','Bought_amount','Sold_amount','net').show(20)

+--------------------+-------------+------------+-------------------+
|               buyer|Bought_amount| Sold_amount|                net|
+--------------------+-------------+------------+-------------------+
|\x0000005c7dc69d4...|    837.48308|    1078.273| 240.78991999999994|
|\x00003e81818a2f1...|   125.698549|  173.522188|          47.823639|
|\x0005bb8bdcbbf4a...|     775.9004|   1133.2491|           357.3487|
|\x00079e801ad1382...|    178.37568|   1289.0346| 1110.6589199999999|
|\x000cd27f10dffac...|     1405.935|   55277.167|          53871.232|
|\x000cef8f0ab0a2f...|    694.86968|1094.2062762| 399.33659620000003|
|\x000e0ed8a7307e1...|      905.931|   519.35032|-386.58068000000003|
|\x00110e09134aec0...|   200.720832|   188.68224|-12.038591999999994|
|\x00110e09134aec0...|   200.720832|  250.395432|            49.6746|
|\x00110e09134aec0...|   200.720832|   278.09376|  77.37292799999997|
|\x00110e09134aec0...|   200.720832|  248.424224|  47.70339200000001|
|\x00110e09134aec0..

# Ranking 1 - average amount spent (here profit , loss automatically done)

In [21]:
df_avg = df_all.groupBy("buyer").agg(avg("net").alias("avg_spent"))

df_avg.show(20)


23/03/25 22:04:50 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/03/25 22:04:50 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/03/25 22:04:50 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/03/25 22:04:50 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/03/25 22:04:50 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/03/25 22:04:50 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/03/25 22:04:50 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/03/25 22:04:50 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/03/25 22:04:51 WARN RowBasedKeyValueBatch: Calling spill() on

+--------------------+-------------------+
|               buyer|          avg_spent|
+--------------------+-------------------+
|\x006b82530c7985f...|-252.26791416666666|
|\x014fd068938fa19...|   89.9409247368421|
|\x01981e7284ade33...|-128.70024377777767|
|\x0312b9e6627d9a4...|  1866.865156666667|
|\x05379b307e6ae02...| -80.20050999999994|
|\x05e547449cff394...|-58.825013999999996|
|\x06085d8c8d119a8...|         41.0816605|
|\x0979ce762b7433f...|   135.453857622389|
|\x0a52279e62a5b7a...| 42.250928499999965|
|\x0d2a2a3a5c5594d...| 3093.0049499999996|
|\x0d3e48a5dafd304...| -912.7935540000001|
|\x0dd78af5e9f7c8e...| -135.1928243333333|
|\x100629985d04d8b...|  683.0129499999998|
|\x1036cad2d919634...|         -270.89558|
|\x11fa47616973db7...| -76.56424711111113|
|\x1209931408ff5cb...|-235.36471799999998|
|\x16129dbd51d01f5...|          204.29512|
|\x1773fb4569491cc...| 323.37379599999997|
|\x19b9d5893e5c25f...|           841.2657|
|\x1a910629f688471...|  414.0312807333333|
+----------

# Ranking 2 - number of transactions

In [22]:
df_count = df_all.groupBy("buyer").agg(count("*").alias("number_txns"))

df_count.show(20)

23/03/25 22:05:02 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/03/25 22:05:02 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/03/25 22:05:02 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/03/25 22:05:02 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/03/25 22:05:02 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/03/25 22:05:02 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/03/25 22:05:02 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/03/25 22:05:02 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/03/25 22:05:03 WARN RowBasedKeyValueBatch: Calling spill() on

+--------------------+-----------+
|               buyer|number_txns|
+--------------------+-----------+
|\x006b82530c7985f...|         12|
|\x014fd068938fa19...|         19|
|\x01981e7284ade33...|         18|
|\x0312b9e6627d9a4...|          6|
|\x03a3da04bd4a888...|          1|
|\x05379b307e6ae02...|          3|
|\x05e547449cff394...|          1|
|\x06085d8c8d119a8...|          2|
|\x0979ce762b7433f...|          3|
|\x0a52279e62a5b7a...|          6|
|\x0d2a2a3a5c5594d...|          2|
|\x0d3e48a5dafd304...|          1|
|\x0dd78af5e9f7c8e...|          6|
|\x100629985d04d8b...|          2|
|\x1036cad2d919634...|          1|
|\x105a38654e6c66d...|          5|
|\x11fa47616973db7...|          9|
|\x1209931408ff5cb...|          4|
|\x16129dbd51d01f5...|          2|
|\x1773cb182cd9460...|          4|
+--------------------+-----------+
only showing top 20 rows



# Diversity Ranking 3 - Number of Nft contract address held

In [23]:
df_num_nftcontract = df_all.groupBy("buyer").agg(count("nft_contract_address").alias("num_nftcontract"))

df_num_nftcontract.show(20)

23/03/25 22:05:13 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/03/25 22:05:13 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/03/25 22:05:13 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/03/25 22:05:13 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/03/25 22:05:13 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/03/25 22:05:13 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/03/25 22:05:13 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/03/25 22:05:13 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/03/25 22:05:13 WARN RowBasedKeyValueBatch: Calling spill() on

+--------------------+---------------+
|               buyer|num_nftcontract|
+--------------------+---------------+
|\x006b82530c7985f...|             12|
|\x014fd068938fa19...|             19|
|\x01981e7284ade33...|             18|
|\x0312b9e6627d9a4...|              6|
|\x03a3da04bd4a888...|              1|
|\x05379b307e6ae02...|              3|
|\x05e547449cff394...|              1|
|\x06085d8c8d119a8...|              2|
|\x0979ce762b7433f...|              3|
|\x0a52279e62a5b7a...|              6|
|\x0d2a2a3a5c5594d...|              2|
|\x0d3e48a5dafd304...|              1|
|\x0dd78af5e9f7c8e...|              6|
|\x100629985d04d8b...|              2|
|\x1036cad2d919634...|              1|
|\x105a38654e6c66d...|              5|
|\x11fa47616973db7...|              9|
|\x1209931408ff5cb...|              4|
|\x16129dbd51d01f5...|              2|
|\x1773cb182cd9460...|              4|
+--------------------+---------------+
only showing top 20 rows



# Diversity Ranking 3 - Number of unique currencies transaction done in

In [24]:
unique_names = df_all.select(col("original_currency")).distinct().rdd.flatMap(lambda x: x).collect()

In [25]:
# df_currency = df_all.groupBy("buyer").agg(countDistinct("original_currency").alias("num_currency")).orderBy(desc("num_currency"))
df_currency = df_all.groupBy("buyer").agg(countDistinct("original_currency").alias("num_currency"))
df_currency.show(20)

+--------------------+------------+
|               buyer|num_currency|
+--------------------+------------+
|\x375751f0159c9bb...|           1|
|\x3bd12ee645be873...|           1|
|\xf9e5db061035038...|           1|
|\x1773fb4569491cc...|           1|
|\x890652186e45f7b...|           1|
|\xc5296880f66398a...|           1|
|\x337febbfe19c7eb...|           1|
|\xf79871fbef70124...|           1|
|\x0dd78af5e9f7c8e...|           1|
|\x7e3e126ee75b008...|           1|
|\x83ac460d910b2cc...|           1|
|\x5c41d062f78a645...|           1|
|\x898bacc2563152c...|           1|
|\xcf2f2a4876c294e...|           1|
|\x87aa2feb827c7b1...|           1|
|\x58363b57f926039...|           1|
|\x05379b307e6ae02...|           1|
|\x11fa47616973db7...|           1|
|\x72f12257b468f62...|           1|
|\x41690d2a93af3b9...|           1|
+--------------------+------------+
only showing top 20 rows



In [26]:
# data_collect = df_all.collect()

# Diversity Ranking 3 - Average Duration (in seconds)

In [27]:

df_all = df_all.withColumn("duration", (unix_timestamp(col("Sold_Timestamp")) - unix_timestamp(col("Bought_Timestamp"))))

In [28]:
df_all.select('duration').show(5)

+--------+
|duration|
+--------+
|  298840|
|   37944|
|   91081|
| 4252760|
| 6705652|
+--------+
only showing top 5 rows



In [29]:
df_avg_duration = df_all.groupBy("buyer").agg(avg("duration").alias("avg_duration"))

df_avg_duration.show(20)

23/03/25 22:05:51 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/03/25 22:05:51 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/03/25 22:05:51 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/03/25 22:05:51 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/03/25 22:05:51 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/03/25 22:05:51 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/03/25 22:05:51 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/03/25 22:05:51 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/03/25 22:05:51 WARN RowBasedKeyValueBatch: Calling spill() on

+--------------------+------------------+
|               buyer|      avg_duration|
+--------------------+------------------+
|\x006b82530c7985f...|3849957.9166666665|
|\x014fd068938fa19...|         1528443.0|
|\x01981e7284ade33...| 842066.3888888889|
|\x0312b9e6627d9a4...|2391314.6666666665|
|\x03a3da04bd4a888...|             828.0|
|\x05379b307e6ae02...|169497.33333333334|
|\x05e547449cff394...|         7766736.0|
|\x06085d8c8d119a8...|          260927.5|
|\x0979ce762b7433f...| 529839.6666666666|
|\x0a52279e62a5b7a...|1752676.8333333333|
|\x0d2a2a3a5c5594d...|         4069915.5|
|\x0d3e48a5dafd304...|          899052.0|
|\x0dd78af5e9f7c8e...| 395363.6666666667|
|\x100629985d04d8b...|           61503.5|
|\x1036cad2d919634...|         6738024.0|
|\x105a38654e6c66d...|         7016526.0|
|\x11fa47616973db7...|155996.44444444444|
|\x1209931408ff5cb...|           10509.5|
|\x16129dbd51d01f5...|          568803.5|
|\x1773cb182cd9460...|           57996.5|
+--------------------+------------

In [30]:
type(df_avg_duration)

pyspark.sql.dataframe.DataFrame

# Feature Extraction for Model

In [31]:
# modeldata = reduce(lambda x,y: pd.merge(x,y, on='buyer', how='outer'), [df_avg, df_count, df_num_nftcontract, df_currency, df_avg_duration])
joined_df = df_avg.join(df_count, on='buyer', how='inner').join(df_num_nftcontract, on='buyer', how='inner').join(df_currency, on='buyer', how='inner').join(df_avg_duration, on='buyer', how='inner')

In [32]:
joined_df.show(5)

23/03/25 22:06:11 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/03/25 22:06:11 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/03/25 22:06:11 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/03/25 22:06:11 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/03/25 22:06:11 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/03/25 22:06:11 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/03/25 22:06:11 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/03/25 22:06:12 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/03/25 22:06:12 WARN RowBasedKeyValueBatch: Calling spill() on

23/03/25 22:06:20 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/03/25 22:06:20 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/03/25 22:06:20 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/03/25 22:06:20 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/03/25 22:06:20 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/03/25 22:06:21 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/03/25 22:06:21 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/03/25 22:06:21 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/03/25 22:06:22 WARN RowBasedKeyValueBatch: Calling spill() on

+--------------------+-------------------+-----------+---------------+------------+------------------+
|               buyer|          avg_spent|number_txns|num_nftcontract|num_currency|      avg_duration|
+--------------------+-------------------+-----------+---------------+------------+------------------+
|\x000000000027634...|-1094.1415000000002|          2|              2|           1|         1012977.0|
|\x0000002b94395a5...| 18.830347000000003|          1|              1|           1|           53640.0|
|\x00052fb2f4c461b...|         5686.01735|          2|              2|           2|         1696346.0|
|\x0005bb8bdcbbf4a...| 288.20619921428573|          7|              7|           1|256193.85714285713|
|\x0007a07be3fc1a3...|-305.52065600000003|          1|              1|           1|          208449.0|
+--------------------+-------------------+-----------+---------------+------------+------------------+
only showing top 5 rows



In [33]:
joined_df.count()

186035

In [34]:
joined_df.write.parquet("/Users/yashasvi/Documents/Model/")

23/03/25 22:07:06 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/03/25 22:07:06 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/03/25 22:07:06 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/03/25 22:07:06 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/03/25 22:07:07 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/03/25 22:07:07 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/03/25 22:07:07 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/03/25 22:07:07 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/03/25 22:07:07 WARN RowBasedKeyValueBatch: Calling spill() on

23/03/25 22:07:14 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/03/25 22:07:14 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/03/25 22:07:14 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/03/25 22:07:14 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/03/25 22:07:14 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/03/25 22:07:14 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/03/25 22:07:14 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/03/25 22:07:14 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/03/25 22:07:14 WARN RowBasedKeyValueBatch: Calling spill() on

23/03/25 22:07:30 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 96.54% for 7 writers
